# 03 - Model training and deployment using SageMaker

This notebook will cover the data modeling and deployment using AWS resources

## Table of Contents
* [1. Package & Data Import](#chapter1)
* [2. Upload Data to S3](#chapter2)
    * [2.1 Prepare Data](#chapter2.1)
    * [2.2 Uploading training data](#chapter2.2)
* [3. Model Training and Deployment](#chapter3)
    * [3.1. Train base model](#chapter3.2)
    * [3.2. Deploy and evaluate base model](#chapter3.2)
    * [3.3 Main model - LSTM Neural Network](#chapter3.3)
* [4. Train/Test Split](#chapter4)
* [5. Feature Engineering](#chapter5)
* [6. Save data for Modeling](#chapter6)

### 1. Package & Data Import <a class="anchor" id="chapter1"></a>
***

In [1]:
# Basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3

import os

In [2]:
!pip install sagemaker==1.72.0

# Sagemaker
import sagemaker

     |████████████████████████████████| 297 kB 6.3 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=386358 sha256=9482184761f9e27e0e661776ec9083f62b2f63eafa1cfe76da484586e774d682
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.38.0
    Uninstalling sagemaker-2.38.0:
      Successfully uninstalled sagemaker-2.38.0


In [3]:
X_train = pd.read_csv('../data/xtrain.csv', header = None)
X_test =  pd.read_csv('../data/xtest.csv', header = None)

y_train = pd.read_csv('../data/ytrain.csv', header = None)
y_test =  pd.read_csv('../data/ytest.csv', header = None)

In [4]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,183,44,1171,23,763,44,1415,454,1,23,...,0,0,0,0,0,0,0,0,0,0
1,131,166,166,13,166,166,22,433,414,5,...,0,0,0,0,0,0,0,0,0,0
2,101,21,42,33,16,12,120,1422,1578,401,...,0,0,0,0,0,0,0,0,0,0
3,161,1569,600,1,663,1562,142,320,172,28,...,0,0,0,0,0,0,0,0,0,0
4,64,11,238,44,330,633,1680,978,61,1,...,0,0,0,0,0,0,0,0,0,0


### 2. Upload data to S3
***

In [5]:
assert len(y_train) == len(X_train)
#len(X_train)

In [6]:
data_dir = '../data/'

#### 2.1 Prepare Data


In [7]:
aws_data = pd.concat([y_train, X_train], axis=1)

aws_data.to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [8]:
aws_data.head()

,0,0,1,2,3,4,5,6,7,8,...,491,492,493,494,495,496,497,498,499,500
0,0,52,7,93,819,3326,269,201,1856,180,...,0,0,0,0,0,0,0,0,0,0
1,0,69,1368,122,1275,2,2577,758,186,391,...,0,0,0,0,0,0,0,0,0,0
2,0,60,994,894,637,849,2152,69,51,40,...,0,0,0,0,0,0,0,0,0,0
3,0,134,7,11,51,99,3330,48,26,4,...,0,0,0,0,0,0,0,0,0,0
4,1,88,1512,1049,316,27,4,178,308,688,...,0,0,0,0,0,0,0,0,0,0


#### 2.2. Uploading training data

In [9]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [10]:
data_dir = "../data"
prefix = 'positiveness-lyrics'

# upload all data to S3
data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(data)

s3://sagemaker-us-east-2-890904620905/positiveness-lyrics


In [11]:
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

positiveness-lyrics/train.csv
positiveness-lyrics/word_dict.pkl
positiveness-lyrics/xtest.csv
positiveness-lyrics/xtrain.csv
positiveness-lyrics/ytest.csv
positiveness-lyrics/ytrain.csv
Test passed!


### 3. Model Training
***

#### 3.1 Train Base Model - naiveBayes

In [52]:
# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn

output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate our custom SKLearn estimator
estimator = SKLearn(entry_point='train.py',
                    source_dir='./utilis/bayes_model',
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    py_version='py3',
                    framework_version='0.23-1',
                    sagemaker_session=sagemaker_session
                   
                   )

In [53]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': data})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2021-05-04 18:50:37 Starting - Starting the training job...
2021-05-04 18:50:39 Starting - Launching requested ML instances......
2021-05-04 18:51:42 Starting - Preparing the instances for training.........
2021-05-04 18:53:33 Downloading - Downloading input data
2021-05-04 18:53:33 Training - Downloading the training image...
2021-05-04 18:54:02 Uploading - Uploading generated training model
2021-05-04 18:54:02 Completed - Training job completed
2021-05-04 18:53:50,393 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-05-04 18:53:50,396 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-04 18:53:50,405 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-05-04 18:53:50,784 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-04 18:53:52,210 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-04 18:53:

#### 3.2. Deploy and evaluate base model

In [54]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------------!

In [50]:
type(X_test)

pandas.core.frame.DataFrame

In [70]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# generate
y_preds  = predictor.predict(X_test)
#print(y_preds)
accuracy = accuracy_score([round(num) for num in y_preds[:,1]], y_test)
auc = roc_auc_score(y_test, y_preds[:,1])

print(f"The estimator had an auc of {auc:.2%} and an accruacy of {accuracy:.2%} in the test set")

The estimator had an auc of 55.32% and an accruacy of 51.00% in the test set


In [66]:
pd.DataFrame(y_preds).to_csv("./bayes_preds.csv", index=False)

In [71]:
#clean endpoint
boto3.client('sagemaker').delete_endpoint(EndpointName=estimator.endpoint)

AttributeError: 'SKLearn' object has no attribute 'endpoint'

#### 3.3 Main model - LSTM Neural Network

In [28]:
import torch
import torch.utils.data

train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [29]:
train_sample

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,0,52,7,93,819,1,269,201,1,180,...,0,0,0,0,0,0,0,0,0,0
1,0,69,1,122,1,2,1,758,186,391,...,0,0,0,0,0,0,0,0,0,0
2,0,60,994,894,637,849,1,69,51,40,...,0,0,0,0,0,0,0,0,0,0
3,0,134,7,11,51,99,1,48,26,4,...,0,0,0,0,0,0,0,0,0,0
4,1,88,1,1,316,27,4,178,308,688,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,144,1,1,794,885,813,1,1,57,...,0,0,0,0,0,0,0,0,0,0
796,1,254,10,53,53,53,343,12,53,53,...,0,0,0,0,0,0,0,0,0,0
797,1,56,676,326,78,1,35,207,61,814,...,0,0,0,0,0,0,0,0,0,0
798,1,271,1,1,1,181,4,217,341,1,...,0,0,0,0,0,0,0,0,0,0


In [30]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # clear gradiants
            optimizer.zero_grad()
            
            # compute the model output
            out = model(batch_X)
            
            # calculate loss
            loss = loss_fn(out, batch_y)
            
            # backward propagation
            loss.backward()
            
            # update model weights with optimizer
            optimizer.step()
            
            total_loss += loss.data.item()
        print(f"Epoch: {epoch}, BCELoss: {total_loss / len(train_loader)}")

In [31]:
import torch.optim as optim
from utilis.lstm_model.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6898287311196327
Epoch: 2, BCELoss: 0.6772803962230682
Epoch: 3, BCELoss: 0.6649945974349976
Epoch: 4, BCELoss: 0.6455747671425343
Epoch: 5, BCELoss: 0.6209660843014717


## Train model

In [16]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="./utilis/lstm_model",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 20,
                        'hidden_dim': 200,
                        'vocab_size': 4000,
                        'seed': 123,
                    })

In [17]:
estimator.fit({'training': data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-05-03 22:47:23 Starting - Starting the training job...
2021-05-03 22:47:25 Starting - Launching requested ML instances......
2021-05-03 22:48:29 Starting - Preparing the instances for training...
2021-05-03 22:49:22 Downloading - Downloading input data...
2021-05-03 22:49:53 Training - Downloading the training image...
2021-05-03 22:50:14 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-03 22:50:16,017 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-05-03 22:50:16,020 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-03 22:50:16,032 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-03 22:50:17,492 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-03 22:50:17,786 sagemaker-containers INFO  

In [34]:
project_predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

## Test Model

In [35]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, project_predictor.predict(array))
    
    return predictions

In [56]:
predictions = predict(X_test.values)
predictions_label = [round(num) for num in predictions]

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions_label)

0.52

In [58]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

0.5148514851485149

## test individual lyric

In [50]:
import pickle
with open("../data/word_dict.pkl", "rb") as f:
        word_dict = pickle.load(f)

In [38]:
test = "Imagine there's no heaven It's easy if you try No hell below us Above us only sky Imagine all the people Living for today"

In [39]:
!pip install nltk

In [46]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

nltk.download('stopwords')
nltk.download('words')

def lyrics_to_words(lyrics):
    '''
    helper function to clean out song lyrics. We apply porter Stemmer algorithm and remove stopwords
    '''
    stopwords = nltk.corpus.stopwords.words('english')
    newStopWords = ['verse','1', '2', 'chorus', 'bridge', 'talking', 'refrain', 'explain', 'request']
    stopwords.extend(newStopWords)
    stemmer = PorterStemmer()
    
    words_english = set(nltk.corpus.words.words())

    remove_non_english = " ".join(w for w in nltk.wordpunct_tokenize(lyrics) if w.lower() in words_english or not w.isalpha())
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", remove_non_english.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [47]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

In [51]:
process_data = lyrics_to_words(test)

data_processed, length = convert_and_pad(word_dict, process_data)

test_data = np.append(length, np.array(data_processed)).reshape(1,-1) # add the length in the beggining

In [52]:
project_predictor.predict(test_data)

array(0.6079634, dtype=float32)

In [59]:
project_predictor.delete_endpoint()

### 4. Deploy the model for the web app

In [18]:
estimator

In [20]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='./utilis/lstm_model',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-----------------------------*

UnexpectedStatusException: Error hosting endpoint sagemaker-pytorch-2021-05-03-23-00-28-687: Failed. Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

## Clean up Resources


In [17]:
project_predictor.endpoint

NameError: name 'project_predictor' is not defined

In [58]:
boto3.client('sagemaker').delete_endpoint(EndpointName=project_predictor.endpoint)


{'ResponseMetadata': {'RequestId': 'fbb6e7c9-6a14-44e4-99f2-74145bce0eac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fbb6e7c9-6a14-44e4-99f2-74145bce0eac',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 02 May 2021 18:15:37 GMT'},
  'RetryAttempts': 0}}

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()